# Run this code to deploy an agent

In [2]:
import vertexai
from virtual_nurse.agent import root_agent

PROJECT_ID = "virtual-nurse-450613"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://deploy-virtual-nurse"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

session = app.create_session(user_id="u_123")

In [3]:
session

Session(id='c9339495-3937-4efe-9694-9cedbb5495d8', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1750540169.2280614)

In [4]:
app.list_sessions(user_id="u_123")

ListSessionsResponse(sessions=[Session(id='c9339495-3937-4efe-9694-9cedbb5495d8', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1750540169.2280614)])

In [ ]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="Scarlett, age 10, had a tonsillectomy 5 days ago and is currently visiting cousins out of town. The parent reports that the son's mouth smells really bad and he says it tastes gross. The patient does not report bleeding. The distance from the surgical hospital is 45 minutes and from the nearest hospital is 20 minutes. The patient last ate this afternoon and drank 1.5 cups of fluids in the past day. Lips are not chapped. The patient is not acting like themselves\u2014complaining a lot. Pain is not limiting intake. The patient is not experiencing nausea or vomiting and has no fevers. Pain is reported in the throat. There is no ear pain. The patient has a sore throat. Pain is managed with ibuprofen. Medications are not being alternated and they have not run out. Pain is not impacting intake. There is no bloody sputum. The throat has white patches only. The tongue/mouth has a yellowish coating. There is no breathing history and no cyanosis. The patient is annoyed and avoidant and not acting normal.",
):
    print(event)

In [4]:
from vertexai import agent_engines

In [6]:
remote_app = agent_engines.create(
    agent_engine=root_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]", "llama-index"  
    ],
    extra_packages=[
        "virtual_nurse",  
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.97.0', 'pydantic': '2.11.7'}
The following requirements are missing: {'cloudpickle', 'pydantic'}
The following requirements are appended: {'pydantic==2.11.7', 'cloudpickle==3.1.1'}
The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'llama-index', 'pydantic==2.11.7', 'cloudpickle==3.1.1']
Using bucket deploy-virtual-nurse
Wrote to gs://deploy-virtual-nurse/agent_engine/agent_engine.pkl
Writing to gs://deploy-virtual-nurse/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://deploy-virtual-nurse/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/533873564462/locations/us-central1/reasoningEngines/6714699918641987584/operations/7493935732076052480
View progress and logs at https://console.cloud.google.com/logs/query?project=virtual-nurs

In [7]:
remote_app.resource_name

'projects/533873564462/locations/us-central1/reasoningEngines/6714699918641987584'

In [5]:
resource_id = "6714699918641987584"
user_id = "hoan_123"
agent = agent_engines.get(resource_id)
print(f"Found agent with resource ID: {resource_id}")
session = agent.create_session(user_id=user_id)
print(f"Created session for user ID: {user_id}")
print("Type 'quit' to exit.")
while True:
    user_input = input("Input: ")
    print(f"User input: {user_input}")
    if user_input == "quit":
        break

    for event in agent.stream_query(
        user_id=user_id, session_id=session["id"], message=user_input
    ):
        if "content" in event:
            if "parts" in event["content"]:
                parts = event["content"]["parts"]
                for part in parts:
                    if "text" in part:
                        text_part = part["text"]
                        print(f"Response: {text_part}")

agent.delete_session(user_id=user_id, session_id=session["id"])
print(f"Deleted session for user ID: {user_id}")

Found agent with resource ID: 6714699918641987584
Created session for user ID: hoan_123
Type 'quit' to exit.
User input: 
Response: Hello! I'm here to help you with concerns about your child's recovery after their tonsillectomy. I need to ask a series of questions to understand the situation fully and provide the best guidance.

First, what is your child's name?
User input: Scarlett
Response: Thank you. What is Scarlett's age?
User input: 10
Response: Got it. What surgery did Scarlett have?
User input: had a tonsillectomy 5 days ago and is currently visiting cousins out of town. The parent reports that the son's mouth smells really bad and he says it tastes gross. The patient does not report bleeding. The distance from the surgical hospital is 45 minutes and from the nearest hospital is 20 minutes. The patient last ate this afternoon and drank 1.5 cups of fluids in the past day. Lips are not chapped. The patient is not acting like themselves\u2014complaining a lot. Pain is not limiting

In [8]:
remote_session = remote_app.create_session(user_id="u_123")
remote_session

{'appName': '6714699918641987584',
 'id': '6717818683374174208',
 'state': {},
 'userId': 'u_123',
 'events': [],
 'lastUpdateTime': 1750541275.465684}

In [72]:
for event in remote_app.stream_query(
    user_id="u_123",
    session_id=remote_session["id"],
    message="""Scarlett, age 10, had a tonsillectomy 5 days ago and is 
    currently visiting cousins out of town. The parent reports that the son's 
    mouth smells really bad and he says it tastes gross. The patient does not 
    report bleeding. The distance from the surgical hospital is 45 minutes and 
    from the nearest hospital is 20 minutes. The patient last ate this afternoon 
    and drank 3 cups of fluids in the past day. Lips are not chapped. The patient 
    is not acting like themselves,complaining a lot. Pain is not limiting intake. 
    The patient is not experiencing nausea or vomiting and has no fevers. Pain is reported 
    in the throat. There is no ear pain. The patient has a sore throat. Pain is 
    managed with ibuprofen. Medications are not being alternated and they have not 
    run out. Pain is not impacting intake. There is no bloody sputum. The throat has 
    white patches only. The tongue/mouth has a yellowish coating. There is no breathing 
    history and no cyanosis. The patient is annoyed and avoidant and not acting normal.""",
):
    print(event)

{'content': {'parts': [{'text': "Here is a summary of the information you provided:\n\nPatient name: Scarlett\nPatient age: 10 years old\nSurgery type: Tonsillectomy\nWhen surgery happened: 5 days ago\nCurrent location: Out of town, visiting cousins\nBleeding reported: No\nBreathing OK: Yes\nDistance from surgical hospital: 45 minutes\nDistance from the nearest hospital: 20 minutes\nWhen the patient last ate: This afternoon\nDrink amount in the past day: 1.5 cups\nLips chapped/appearance: Lips are not chapped.\nActing like themselves or not: Not acting like themselves—complaining a lot.\nPain is preventing them from eating and drinking: No\nNausea or vomiting: No\nFever or not: No fever\nPain or not: Yes\nIf pain, pain location: Throat\nIf pain, pain management regimen: Ibuprofen\nBloody sputum or not: No\nHealing issues or not: Yes\nIf there is healing issues, how the throat looks: White patches only.\nIf there is healing issues, what else you see: Yellowish coating on tongue/mouth.\n